In [2]:
import pandas as pd
import numpy as np
import json
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Do not truncate column content
pd.set_option('display.width', 1000)        # Prevent wrapping of wide DataFrames

### Generating urls and gids for the shared google sheet tabs

In [ ]:
# Path to your service account JSON
SERVICE_ACCOUNT_FILE = 'C:/Users/511232/Desktop/DSS/Consolidated questionnaires/online questionnnaires/credentials_serviceaccount.json'

# Set main folder ID
MAIN_FOLDER_ID = '1nZ8l69GkB6EQQPoiOF-6CEVx2q3vAhcU'

# Step 1: List all country subfolders
query = f"'{MAIN_FOLDER_ID}' in parents and mimeType='application/vnd.google-apps.folder'"
response = drive_service.files().list(q=query, fields="files(id, name)").execute()
country_folders = response.get('files', [])

results = []

# Step 2: For each country folder

keywords=['Health', 'Education','Housing', 'Population', 'Labor', 'Poverty']

for folder in country_folders:
    folder_id = folder['id']
    folder_name = folder['name']
    
    # Step 3: List Google Sheets in the country folder
    query_files = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"
    response_files = drive_service.files().list(q=query_files, fields="files(id, name)").execute()
    files = response_files.get('files', [])

    for file in files:
        file_id = file['id']
        file_name = file['name']
        
        # Step 4: Get sheet tabs and gids
        sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId=file_id).execute()
        sheets = sheet_metadata.get('sheets', [])

        for sheet in sheets:
            tab_name = sheet['properties']['title']

            if any(keyword.lower() in tab_name.lower() for keyword in keywords):

                gid = sheet['properties']['sheetId']
                url = f"https://docs.google.com/spreadsheets/d/{file_id}/edit#gid={gid}"

                results.append({
                    'country_folder': folder_name,
                    'file_name': file_name,
                    'tab_name': tab_name,
                    'gid': gid,
                    'url': url
                })

# Display results
df = pd.DataFrame(results)
print(df)
df.to_csv('sheet_tabs_urls_all_countries.csv', index=False)


In [12]:
#for Algeria alone

# Path to your service account JSON
SERVICE_ACCOUNT_FILE = 'C:/Users/511232/Desktop/DSS/Consolidated questionnaires/online questionnnaires/credentials_serviceaccount.json'

SCOPES = [
    'https://www.googleapis.com/auth/drive.readonly',
    'https://www.googleapis.com/auth/spreadsheets.readonly'
]

FOLDER_ID = '1ftIKuPB_bnSKPiBNLblOtaz-11WQDUwq'  # Algeria folder ID

# -----------------------------
# AUTHENTICATE
# -----------------------------
creds = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

drive_service = build('drive', 'v3', credentials=creds)
sheets_service = build('sheets', 'v4', credentials=creds)

# -----------------------------
# LIST GOOGLE SHEETS IN ALGERIA
# -----------------------------
query = f"'{FOLDER_ID}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"
response = drive_service.files().list(q=query, fields="files(id, name)").execute()
files = response.get('files', [])

results = []

keywords=['Health', 'Education','Housing', 'Population', 'Labor', 'Poverty']

for file in files:
    file_id = file['id']
    file_name = file['name']

    # Get sheet tabs
    sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId=file_id).execute()
    sheets = sheet_metadata.get('sheets', [])

    for sheet in sheets:
        tab_name = sheet['properties']['title']

        # Filter tabs with required keywords
        if any(keyword.lower() in tab_name.lower() for keyword in keywords):
            gid = sheet['properties']['sheetId']
            url = f"https://docs.google.com/spreadsheets/d/{file_id}/edit#gid={gid}"

            results.append({
                'file_name': file_name,
                'tab_name': tab_name,
                'gid': gid,
                'url': url
            })

# Display results
df = pd.DataFrame(results)
df.head()


                                      file_name         tab_name         gid                                                                                                      url
0    Algeria Housing Questionnaire_updated_2025        Housing_1  1096513440  https://docs.google.com/spreadsheets/d/1LCB2atXeECpkib6GSfHE6MrsNU0E7KeqAEYVik_YNgo/edit#gid=1096513440
1    Algeria Housing Questionnaire_updated_2025        Housing_2  1288519366  https://docs.google.com/spreadsheets/d/1LCB2atXeECpkib6GSfHE6MrsNU0E7KeqAEYVik_YNgo/edit#gid=1288519366
2    Algeria Housing Questionnaire_updated_2025        Housing_3    83882750    https://docs.google.com/spreadsheets/d/1LCB2atXeECpkib6GSfHE6MrsNU0E7KeqAEYVik_YNgo/edit#gid=83882750
3    Algeria Housing Questionnaire_updated_2025        Housing_4   871806362   https://docs.google.com/spreadsheets/d/1LCB2atXeECpkib6GSfHE6MrsNU0E7KeqAEYVik_YNgo/edit#gid=871806362
4    Algeria Housing Questionnaire_updated_2025        Housing_5   277130603   https://doc

### read in csvs directly from google drive

### for arabic questionnaires

In [ ]:
# read in the url_gid csv
url_gid=pd.read_csv('C:/Users/511232/Desktop/DSS/Consolidated questionnaires/online questionnnaires/url_gid.csv')

# HOUSING
housing_url_gid=url_gid[url_gid['section']=='Housing'].copy()

housing_dfs=[]

#loop through the dicitonary
for url, gid in zip(housing_url_gid['url'], housing_url_gid['gid']):
    
    full_url = url.format(s=gid)
    df = pd.read_csv(full_url)
    #append to the list of housing dataframes
    housing_dfs.append(df)


In [3]:
housing_dfs[0].head()

,المؤشر,الدولة,المنطقة,نوع مكان الإقامة,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,نوع مكان الإقامة (نسبة مئوية من مجموع المساكن),الجزائر,جميع المناطق,السكن التقليدي مع المرافق الأساسية,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,نوع مكان الإقامة (نسبة مئوية من مجموع المساكن),الجزائر,جميع المناطق,السكن التقليدي لا يحتوي على جميع المرافق الأساسية,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,نوع مكان الإقامة (نسبة مئوية من مجموع المساكن),الجزائر,جميع المناطق,وحدات السكن شبه الدائمة,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,نوع مكان الإقامة (نسبة مئوية من مجموع المساكن),الجزائر,جميع المناطق,وحدات السكن المتنقلة,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,نوع مكان الإقامة (نسبة مئوية من مجموع المساكن),الجزائر,جميع المناطق,وحدات السكن العشوائية,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(housing_dfs)

24

### Translate the datasets

In [47]:
#read in the dictionary excel file and make the English dimensions lower case
path='C:/Users/511232/United Nations/ESCWA-SD - Documents/General/SD/Teams/Demographic and SOCIAL S  Team/0- Compendium - Arab Society/Compendium 2025-2026'
df_translation=pd.read_excel(path+'/translation dict.xlsx')


#create a English to Arabic dictionary
En_Ar_dictionary={}

for dim in [d for d in df_translation['col_en'].unique() if d not in ['year', 'value', 'source']]:
    df_dim=df_translation[df_translation['col_en'].isin([dim.lower(), dim])].copy()
    En_Ar_dictionary.update(
        {dim:{'dim_values':dict(zip(df_dim['val_en'], df_dim['val_ar'])), 
              'dim': {df_dim['col_en'].unique()[0]:df_dim['col_ar'].unique()[0]}}})
    
Ar_En_dictionary={}

for dim in [d for d in df_translation['col_ar'].unique() if d not in ['السنة', 'العدد', 'المصدر']]:
    df_dim=df_translation[df_translation['col_ar'].isin([dim.lower(), dim])].copy()
    Ar_En_dictionary.update(
        {dim:{'dim_values':dict(zip(df_dim['val_ar'], df_dim['val_en'])), 
              'dim': {df_dim['col_ar'].unique()[0]:df_dim['col_en'].unique()[0]}}})


In [48]:
Ar_En_dictionary.keys()

dict_keys(['المؤشر', 'الدولة', 'المواطنة', 'الجنس', 'القطاع', 'المرحلة التعليمية', 'الفئة', 'نوع\xa0الخدمات/المنتجات', 'نوع مكان الإقامة', 'نوع حيازة الوحدات السكنية', 'المنطقة', 'مصدر مياه الشرب', 'أنواع نظام التخلص من مياه الصرف الصحي', 'مصدر الإضاءة', 'الفئة العمرية', 'أسباب البقاء خارج القوى العاملة', 'وضع العمالة', 'أقسام النشاط الإقتصادي', 'القطاع المؤسسي', 'أقسام المهن الرئيسية', 'أسباب الوفيات (نسبة مئوية) (ICD 10)', 'أسباب الوفيات (نسبة مئوية) (ICD 11)', 'الحالة الزوجية'])

In [51]:
class Translator:
    def __init__(self, translate_to, en_ar_dict=None,ar_en_dict=None):
        if translate_to.lower() not in ['english', 'arabic']:
            raise ValueError("Language must be 'english' or 'arabic'")

        self.translate_to = translate_to.lower()

        #Let the user pass their own translation dictionaries if they want, otherwise use the defaults:
        self.en_ar_dict = en_ar_dict if en_ar_dict is not None else En_Ar_dictionary
        self.ar_en_dict = ar_en_dict if ar_en_dict is not None else Ar_En_dictionary

    #translate the column names and values
    def translate(self, df):
        df_translated = df.copy()
        translation_dict = self.ar_en_dict if self.translate_to == 'english' else self.en_ar_dict

        for col, col_dict in translation_dict.items():
            if col in df_translated.columns:
                # Get the translated name
                new_col_name = list(col_dict['dim'].values())[0]  
                print(f'Column "{col}" is matched and translated to {col_dict['dim']}.')
                #rename the column values
                df_translated[col] = df_translated[col].replace(col_dict['dim_values'])
                #rename the column
                df_translated.rename(columns=col_dict['dim'], inplace=True)
                print(f'Translated values: {df_translated[new_col_name].unique()}\n')

        return df_translated

In [ ]:
translator = Translator('english')
df_translated = translator.translate(df_combined)

In [53]:
df_translated.head()

,Country,Nationality,Sex,Indicator,Year,Value,Source,Sector,Education level
0,Somalia,Nationality Total,Male,Net enrolment rate in primary education (percent),2010,1,a,NaN,NaN
1,Somalia,Nationality Total,إناث,Net enrolment rate in primary education (percent),2010,1,a,NaN,NaN
2,Somalia,Nationality Total,Both sexes,Net enrolment rate in primary education (percent),2010,1,a,NaN,NaN
3,Somalia,Nationals,Male,Net enrolment rate in primary education (percent),2010,1,a,NaN,NaN
4,Somalia,Nationals,إناث,Net enrolment rate in primary education (percent),2010,1,a,NaN,NaN


#### check schema with pandera and rapidfuzz to match with possible different column names

In [13]:
import pandas as pd
from rapidfuzz import process
import pandera as pa
from pandera import Column, DataFrameSchema, Check
from pandera.errors import SchemaError
from textwrap import fill

In [14]:
STANDARD_SCHEMA = {
'Indicator': ['indicator','Ind','مؤشر','المؤشر'],
'Country': ['الدولة','البلد','country'],
'Nationality': ['المواطنة','المواطنية','nationality','nationalities'],
'Sex': ['sex','gender','الجنس'],
'Year': ['year','time','Time','سنة','السنة'],
'Value': ['value','number','Number','الرقم','عدد', 'العدد'],
'Comments': ['comments','notes','ملاحظة','المصدر','Source'],
'Sector': ['sector','القطاع'],
'Education level': ['edu level','Education_level','المرحلة التعليمية','التعليم']
}


In [15]:
#map columns to col names using fuzzy match
col_names = {}
for col, aliases in STANDARD_SCHEMA.items():
    match, score, _ = process.extractOne(col, df_translated.columns.tolist() + aliases)
    
    if score > 80:
        print(f'{match} has a score of {score} for column {col}')
        col_names[match] = col

#rename column names according to schema
df_ar_translated=df_translated.rename(col_names)

df_ar_translated.columns

Ind has a score of 90.0 for column Indicator
country has a score of 85.71428571428572 for column Country
nationality has a score of 90.9090909090909 for column Nationality
Year has a score of 100.0 for column Year
Value has a score of 100.0 for column Value
comments has a score of 87.5 for column Comments
sector has a score of 83.33333333333334 for column Sector
Education_level has a score of 93.33333333333333 for column Education level


Index(['الدولة', 'المواطنة', 'الجنس', 'المؤشر', 'Year', 'Value', 'Source', 'القطاع', 'المرحلة التعليمية'], dtype='object')

#### validate the dataframe column labels

In [94]:
df_ar_translated.dtypes

Indicator           object
Country             object
Nationality         object
Sex                 object
Year                 int64
Value              float64
Source              object
Sector              object
Education level     object
dtype: object

In [28]:
translation_df=pd.read_excel('translation dict.xlsx')
translation_df.head(2)

,col_en,val_en,col_ar,val_ar
0,Indicator,Net enrolment rate in primary education (percent),المؤشر,معدل الإلتحاق الصافي في مرحلة التعليم الإبتدائي (الأساسي) (نسبة مئوية)
1,Indicator,Net enrolment rate in secondary education (percent),المؤشر,معدل الإلتحاق الصافي في مرحلة التعليم الثانوي (نسبة مئوية)


In [38]:
list(translation_df.loc[translation_df['col_en']=='Education level', 'val_en'].unique())

['Primary', 'Preparatory', 'Secondary', 'Tertiary', 'All levels']

In [ ]:
En_Ar_dictionary.keys()

In [43]:
allowed_indicator_values=list(translation_df.loc[translation_df['col_en']=='Indicator', 'val_en'].unique())
allowed_country_values=list(translation_df.loc[translation_df['col_en']=='Country', 'val_en'].unique())
allowed_nationality_values=list(translation_df.loc[translation_df['col_en']=='Nationality', 'val_en'].unique())
allowed_sex_values=list(translation_df.loc[translation_df['col_en']=='Sex', 'val_en'].unique())
allowed_year_values=[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024]
allowed_sector_values=list(translation_df.loc[translation_df['col_en']=='Sector', 'val_en'].unique())
allowed_edulevel_values=list(translation_df.loc[translation_df['col_en']=='Education level', 'val_en'].unique())

#define schema
schema = DataFrameSchema({
    'Indicator': Column(str, checks=Check.isin(allowed_indicator_values)),
    'Country': Column(str, checks=Check.isin(allowed_country_values)),
    'Nationality': Column(str, checks=Check.isin(allowed_nationality_values),nullable=True),
    'Sex': Column(str, checks=Check.isin(allowed_sex_values),nullable=True),
    'Year': Column(int, checks=Check.isin(allowed_year_values)),
    'Sector': Column(str, checks=Check.isin(allowed_sector_values),nullable=True),
    'Education level': Column(str, checks=Check.isin(allowed_edulevel_values),nullable=True)     
})

#validate the dataframe
try:
    validated_df = schema.validate(df_ar_translated)
except SchemaError as err:
    wrapped_error = fill(str(err.args), width=150)
    print(wrapped_error)

("Column 'Indicator' failed element-wise validator number 0: isin(['Net enrolment rate in primary education (percent)', 'Net enrolment rate in
secondary education (percent)', 'Pupil-teacher ratio (percent)', 'Adult literacy rates', 'Youth literacy rates', 'Expenditure on education as a
percentage of total government expenditure  (percent)', 'Government expenditure on education as a percentage of GDP (percent)']) failure cases: wrong
ind",)


In [42]:
df_ar_translated.loc[0,'Indicator']='wrong ind'
df_ar_translated.head()

,Indicator,Country,Nationality,Sex,Year,Value,Source,Sector,Education level
0,wrong ind,Somalia,Nationality Total,Male,2010,1.0,a,NaN,NaN
1,Net enrolment rate in primary education (percent),Somalia,Nationality Total,NaN,2010,1.0,a,NaN,NaN
2,Net enrolment rate in primary education (percent),Somalia,Nationality Total,Both sexes,2010,1.0,a,NaN,NaN
3,Net enrolment rate in primary education (percent),Somalia,Nationals,Male,2010,1.0,a,NaN,NaN
4,Net enrolment rate in primary education (percent),Somalia,Nationals,NaN,2010,1.0,a,NaN,NaN


In [ ]:
#for english/arabic questionnaires

# f_path = 'C:/Users/511232/Desktop/DSS/Consolidated questionnaires'
# file='Somalia_Education_en.xlsx'

# sheets_dict = pd.read_excel(f_path+'/'+file, sheet_name=None)  # Reads all sheets at once
# print(sheets_dict.keys())

# '''loop through the sheets, isolate the row where theres source, remove it from the dataframe, reshape the dataframe
# repeat the source row equivalent to the non-reshaped dataframe size, append to the df_list and finally concatenate into a single file'''

# df_list=[]

# for key, df in sheets_dict.items():
    
#     print(f'processing sheet {key}')
#     #read in the dataframe
#     df=sheets_dict[key]

#     #clean it from the null rows
#     df1 = df.dropna(how='all')

#     #get the row and column index where source appear
#     source_row_index=df1[df1.isin(['المصدر','Source', 'source']).any(axis=1)].index[0]
#     #column where source appear
#     source_col=df.columns[df1.isin(['المصدر','Source', 'source']).any(axis=0)][0]
#     #get the index of source_col
#     source_col_index=df1.columns.get_loc(source_col)
#     print('Source row and column index are: {source_row_index},{source_col_index}')

#     #separating the dataframe between main dataframe and the source dataframe
#     main_df=df1.iloc[0:source_row_index-1].reset_index(drop=True)
#     source_df=df1.iloc[source_row_index-1 :,source_col_index :].reset_index(drop=True)


#     #action plan: melt main dataframe, transpose the source dataframe and then merge together
#     #get years columns
#     year_columns = [col for col in df.columns if str(col).isdigit()]
#     print(f'years columns are: {year_columns}')

#     id_variables= list(set(main_df.columns) - set(year_columns))

#     df_main_long = pd.melt(main_df, 
#                     id_vars=id_variables,     # columns to keep
#                     value_vars=year_columns,  # columns to unpivot
#                     var_name='Year', 
#                     value_name='Value',
#                     ignore_index=True)

#     #transpose source dataframe
#     source_df_T=source_df.T.reset_index()

#     cols=['Year','Source']
#     #remove the 1st row
#     source_df_T=source_df_T.iloc[1:]
#     source_df_T.columns=cols

#     #merge the 2 dataframes
#     merged_data=pd.merge(df_main_long, source_df_T, on='Year')
#     merged_data.head(3)

#     #append to df_list
#     df_list.append(merged_data)
    
#     #####################################################################################################
    
#     #go through columns and print out the unique  values
#     for col in [c for c in merged_data.columns if c not in ['Source', 'Value']]:
#         print(f'for column {col}: {merged_data[col].unique()}')

#     print('--------------------------------------------------------------\n')
#     ######################################################################################################

# #consolidate into a single file
# df_combined = pd.concat(df_list, axis=0, ignore_index=True)
# #save to csv
# df_combined.to_csv("df_combined.csv", index=False)


dict_keys(['Education_1_a', 'Education_1_b', 'Education_2_a', 'Education_2_b', 'Education_2_c', 'Education_3', 'Education_4', 'Education_5', 'Education_6'])
